# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modelling

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS udacity_mod1_project
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor':1}
        """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity_mod1_project')
except Exception as e:
    print(e)

### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

In [8]:
# try:
#     session.execute("DROP TABLE music_app_history")
# except Exception as e:
#     print(e) 

In [9]:
query = """
        CREATE TABLE IF NOT EXISTS music_app_history (
            sessionId int,
            itemInSession int,
            artist varchar,
            song varchar,
            length double,
            PRIMARY KEY ((sessionId, itemInSession))
            );
"""
# the combination of sessionId and itemInSession make the data unique, so they are my composite key
# this is the recommended approach for when there will be filters on both columns
try:
    session.execute(query)
except Exception as e:
    print(e)            

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """
            INSERT INTO music_app_history (
                sessionId,
                itemInSession,
                artist,
                song,
                length
            )
            VALUES (%s, %s, %s, %s, %s)
        """

        session.execute(
            query,
            (
                int(line[8]),   # sessionId
                int(line[3]),   # itemInSession
                line[0],        # artist
                line[9],        # song
                float(line[5]), # length
            )
        )

In [11]:
query = """
    SELECT *
    FROM music_app_history
    WHERE sessionID=338
        AND itemInSession = 4
"""

df = pd.DataFrame(
    list(session.execute(query)))
df

,sessionid,iteminsession,artist,length,song
0,338,4,Faithless,495.3073,Music Matters (Mark Knight Dub)


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
# try:
#     session.execute("DROP TABLE music_app_history2")
# except Exception as e:
#     print(e) 

In [13]:
query = """
        CREATE TABLE IF NOT EXISTS music_app_history2 (
            userId int,
            sessionId int,
            itemInSession int,
            artist varchar,
            song varchar,
            firstName varchar,
            lastName varchar,
            PRIMARY KEY ((userId, sessionId), itemInSession)
            );
"""
# userid and session id are my composite key - with the addition of itemInSession (clustering column), each row is 
# uniquely identified
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """
            INSERT INTO music_app_history2 (
                userId,
                sessionId,
                itemInSession,
                artist,
                song,
                firstName,
                lastName
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """

        session.execute(
            query,
            (
                int(line[10]),  # userId
                int(line[8]),   # sessionId
                int(line[3]),   # itemInSession
                line[0],        # artist
                line[9],        # song
                line[1],        # firstName
                line[4],        # lastName
            )
        )

In [15]:
query = """
    SELECT artist, song, userId
    FROM music_app_history2
    WHERE userId = 10 
        AND sessionId = 182
        
"""

df = pd.DataFrame(
    list(session.execute(query)))
df

,artist,song,userid
0,Down To The Bone,Keep On Keepin' On,10
1,Three Drives,Greece 2000,10
2,Sebastien Tellier,Kilometer,10
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


### TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [20]:
# try:
#     session.execute("DROP TABLE music_app_history3")
# except Exception as e:
#     print(e) 

In [21]:
query = """
        CREATE TABLE IF NOT EXISTS music_app_history3 (
            song varchar,
            userId int,
            firstName varchar,
            lastName varchar,
            PRIMARY KEY ((song), userId)
            );
"""
# using userId as a clustering column, so that in combination with song (primary key), each row is uniq. identified
try:
    session.execute(query)
except Exception as e:
    print(e)

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """
            INSERT INTO music_app_history3 (
                song,
                userId,
                firstName,
                lastName
            )
            VALUES (%s, %s, %s, %s)
        """

        session.execute(
            query,
            (
                line[9],        # song
                int(line[10]),  # userId
                line[1],        # firstName
                str(line[4])    # lastName
            )
        )

In [23]:
query = """
    SELECT firstName, lastName
    FROM music_app_history3
    WHERE song = 'All Hands Against His Own'
"""

df = pd.DataFrame(
    list(session.execute(query)))
df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [24]:
for i in ['music_app_history', 'music_app_history2', 'music_app_history3']:
    try:
        session.execute("DROP TABLE " + i)
    except Exception as e:
        print(e) 

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()